In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import plotly.graph_objects as go
import h5py
import time

In [2]:
#file = h5py.File('/sdf/group/neutrino/sfogarty/ND_prototype_files/charge_data/module-3/tpc_12-packet-2023_02_22_12_45_CET_module-3_events.h5', 'r')
file = h5py.File('/sdf/group/neutrino/sfogarty/ND_prototype_files/charge_data/module-0/datalog_2021_04_04_00_41_40_CEST_module-0_events_60s_to_360s.h5', 'r')

In [ ]:
large_clusters = file['large_clusters']
small_clusters = file['small_clusters']

chunk_size = 100  # number of events to process in each chunk
n_chunks = (len(large_clusters) + chunk_size - 1) // chunk_size  # number of chunks
PPS_window = 1e3

near_cosmic = np.zeros(len(large_clusters), dtype=np.int32)  # initialize the result array

start = time.time()
for i in range(1):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(large_clusters))
    
    t_min = large_clusters[start_idx:end_idx]['t_min']
    t_max = large_clusters[start_idx:end_idx]['t_max']
    unix = large_clusters[start_idx:end_idx]['unix']
    
    t_small_clusters = small_clusters['t']
    unix_small_clusters = small_clusters['unix']
    
    PPS_min_match = t_min[:, np.newaxis] - PPS_window < t_small_clusters
    PPS_max_match = t_max[:, np.newaxis] + PPS_window > t_small_clusters
    unix_match = unix[:, np.newaxis] == unix_small_clusters
    PPS_match = np.logical_and(PPS_min_match, np.logical_and(PPS_max_match, unix_match))
    
    near_cosmic[start_idx:end_idx] = np.sum(PPS_match, axis=1)

end = time.time()
print('total time = ', end-start)

In [5]:
n_chunks

31